In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from pointnet.dataset import ShapeNetDataset
from pointnet.model import PointNetDenseCls, feature_transform_regularizer
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import easydict

In [2]:
opt = easydict.EasyDict({'model': 'pointnet_part_seg',
                         'batchSize': 2,
                         'nepoch': 50,
                         'workers': 4,
                         'outf': 'seg',
                         'model': '',
                         'dataset': '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/pointnet.pytorch/datasets/shapenetcore_partanno_segmentation_benchmark_v0',
                         'class_choice': 'Chair',
                         'feature_transform': True
                        })

In [3]:
dataset = ShapeNetDataset(
    root=opt.dataset,
    classification=False,
    class_choice=[opt.class_choice])
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=opt.batchSize,
    shuffle=True,
    num_workers=int(opt.workers))

test_dataset = ShapeNetDataset(
    root=opt.dataset,
    classification=False,
    class_choice=[opt.class_choice],
    split='test',
    data_augmentation=False)
testdataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=opt.batchSize,
    shuffle=True,
    num_workers=int(opt.workers))

{'Chair': 0}
{'Airplane': 4, 'Bag': 2, 'Cap': 2, 'Car': 4, 'Chair': 4, 'Earphone': 3, 'Guitar': 3, 'Knife': 2, 'Lamp': 4, 'Laptop': 2, 'Motorbike': 6, 'Mug': 2, 'Pistol': 3, 'Rocket': 3, 'Skateboard': 3, 'Table': 3} 4
{'Chair': 0}
{'Airplane': 4, 'Bag': 2, 'Cap': 2, 'Car': 4, 'Chair': 4, 'Earphone': 3, 'Guitar': 3, 'Knife': 2, 'Lamp': 4, 'Laptop': 2, 'Motorbike': 6, 'Mug': 2, 'Pistol': 3, 'Rocket': 3, 'Skateboard': 3, 'Table': 3} 4


In [4]:
print(len(dataset), len(test_dataset))
num_classes = dataset.num_seg_classes
print('classes', num_classes)

2658 704
classes 4


In [5]:
try:
    os.makedirs(opt.outf)
except OSError:
    pass

blue = lambda x: '\033[94m' + x + '\033[0m'

classifier = PointNetDenseCls(k=num_classes, feature_transform=opt.feature_transform)

if opt.model != '':
    classifier.load_state_dict(torch.load(opt.model))

optimizer = optim.Adam(classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
classifier.cuda()

num_batch = len(dataset) / opt.batchSize

In [6]:
for i, data in enumerate(dataloader, 0):
    print(data)
    break

[tensor([[[ 0.2462, -0.7746,  0.3370],
         [-0.4256, -0.3154, -0.0778],
         [-0.0505,  0.1718, -0.4361],
         ...,
         [-0.4628, -0.3873,  0.3675],
         [ 0.1748,  0.7724, -0.3690],
         [-0.1210, -0.1288,  0.1918]],

        [[ 0.1121, -0.0240, -0.0608],
         [-0.2730, -0.2431,  0.0166],
         [ 0.2525, -0.2199, -0.3668],
         ...,
         [ 0.2974,  0.3101, -0.4968],
         [-0.1383,  0.3908,  0.4650],
         [-0.4591, -0.2665,  0.5556]]]), tensor([[3, 2, 1,  ..., 3, 1, 2],
        [2, 2, 2,  ..., 1, 4, 3]])]


In [7]:
for epoch in range(opt.nepoch):
    scheduler.step()
    for i, data in enumerate(dataloader, 0):
        points, target = data
        points = points.transpose(2, 1)
        points, target = points.cuda(), target.cuda()
        optimizer.zero_grad()
        classifier = classifier.train()
        pred, trans, trans_feat = classifier(points)
        pred = pred.view(-1, num_classes)
        target = target.view(-1, 1)[:, 0] - 1
        #print(pred.size(), target.size())
        loss = F.nll_loss(pred, target)
        if opt.feature_transform:
            loss += feature_transform_regularizer(trans_feat) * 0.001
        loss.backward()
        optimizer.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()
        print('[%d: %d/%d] train loss: %f accuracy: %f' % (epoch, i, num_batch, loss.item(), correct.item()/float(opt.batchSize * 2500)))

        if i % 10 == 0:
            j, data = next(enumerate(testdataloader, 0))
            points, target = data
            points = points.transpose(2, 1)
            points, target = points.cuda(), target.cuda()
            classifier = classifier.eval()
            pred, _, _ = classifier(points)
            pred = pred.view(-1, num_classes)
            target = target.view(-1, 1)[:, 0] - 1
            loss = F.nll_loss(pred, target)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.data).cpu().sum()
            print('[%d: %d/%d] %s loss: %f accuracy: %f' % (epoch, i, num_batch, blue('test'), loss.item(), correct.item()/float(opt.batchSize * 2500)))

    torch.save(classifier.state_dict(), '%s/seg_model_%s_%d.pth' % (opt.outf, opt.class_choice, epoch))

/home/trojan/anaconda3/envs/pointnet/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[0: 0/1329] train loss: 1.522053 accuracy: 0.316200
[0: 0/1329] test loss: 1.391390 accuracy: 0.242600
[0: 1/1329] train loss: 1.429453 accuracy: 0.446600
[0: 2/1329] train loss: 1.291117 accuracy: 0.586200
[0: 3/1329] train loss: 1.118049 accuracy: 0.672400
[0: 4/1329] train loss: 1.338751 accuracy: 0.542200
[0: 5/1329] train loss: 1.295355 accuracy: 0.590800
[0: 6/1329] train loss: 1.457459 accuracy: 0.471200
[0: 7/1329] train loss: 0.999123 accuracy: 0.766000
[0: 8/1329] train loss: 0.968375 accuracy: 0.765800
[0: 9/1329] train loss: 1.025759 accuracy: 0.769000
[0: 10/1329] train loss: 1.021667 accuracy: 0.761200
[0: 10/1329] test loss: 1.064440 accuracy: 0.658600
[0: 11/1329] train loss: 1.222520 accuracy: 0.658200
[0: 12/1329] train loss: 1.196394 accuracy: 0.594400
[0: 13/1329] train loss: 1.113902 accuracy: 0.653200
[0: 14/1329] train loss: 1.356722 accuracy: 0.529400
[0: 15/1329] train loss: 0.826506 accuracy: 0.864800
[0: 16/1329] train loss: 1.279829 accuracy: 0.521200
[0: 17

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f1bb485da70>
Traceback (most recent call last):
  File "/home/trojan/anaconda3/envs/pointnet/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/home/trojan/anaconda3/envs/pointnet/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/trojan/anaconda3/envs/pointnet/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/home/trojan/anaconda3/envs/pointnet/lib/python3.7/multiprocessing/popen_fork.py", line 45, in wait
    if not wait([self.sentinel], timeout):
  File "/home/trojan/anaconda3/envs/pointnet/lib/python3.7/multiprocessing/connection.py", line 921, in wait
    ready = selector.select(timeout)
  File "/home/trojan/anaconda3/envs/pointnet/lib/python3.7/selectors.py", line 415, in se

FileNotFoundError: [Errno 2] No such file or directory